<a href="https://colab.research.google.com/github/merucode/DL/blob/01-colab-study_must_have_pytorch/03-02_basic_regression_house_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

### Install Dependency

In [ ]:
!pip install scikit-learn==1.0.2

# STEP 1. Dataset

Step 1-1. Load data

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

dataset = load_boston() # 데이터셋을 불러옴
print(dataset.keys())

Step 1-2. Check data type

In [ ]:
dataFrame = pd.DataFrame(dataset["data"]) # ❶ 데이터셋의 데이터 불러오기
dataFrame.columns = dataset["feature_names"] # ❷ 특징의 이름 불러오기
dataFrame["target"] = dataset["target"] # ❸ 데이터 프레임에 정답을 추가

print(dataFrame.head()) # ➍ 데이터프레임을 요약해서 출력

## STEP 2. Module

In [ ]:
import torch
import torch.nn as nn

# ❶ 모델 정의
model = nn.Sequential(
   nn.Linear(13, 100),
   nn.ReLU(),
   nn.Linear(100, 1)
)

## STEP 3. Learning

Step 3-1. Setting parameters

In [ ]:
from torch.optim.adam import Adam

X = dataFrame.iloc[:, :13].values # ❷ 정답을 제외한 특징을 X에 입력
Y = dataFrame["target"].values    # 데이터프레임의 target의 값을 추출

batch_size = 100
learning_rate = 0.001

# ❸ 가중치를 수정하기 위한 최적화 정의
optim = Adam(model.parameters(), lr=learning_rate)

Step 3-2. Learning

In [ ]:
# 에포크 반복
for epoch in range(200):

   # 배치 반복
   for i in range(len(X)//batch_size):
       start = i*batch_size      # ➍ 배치 크기에 맞게 인덱스를 지정
       end = start + batch_size

       # 파이토치 실수형 텐서로 변환
       x = torch.FloatTensor(X[start:end])
       y = torch.FloatTensor(Y[start:end])

       optim.zero_grad() # ❺ 가중치의 기울기를 0으로 초기화
       preds = model(x)  # ❻ 모델의 예측값 계산
       loss = nn.MSELoss()(preds, y) # ❼ MSE 손실 계산
       loss.backward()   # 역전파
       optim.step()      # 가중치 업데이트

   if epoch % 20 == 0:
       print(f"epoch{epoch} loss:{loss.item()}")

## STEP 4. Evaluation

In [ ]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction:{prediction.item()} real:{real}")